##### **OPS**

In [ ]:
# CHECK GPU

!nvidia-smi -L

In [ ]:
# MOUNT GOOGLE DRIVE

from google.colab import drive
drive.mount('/content/gdrive')
# All files are no accessible with path /content/gdrive/My Drive/

In [1]:
# ALL IMPORTS

!pip install h-transformer-1d --upgrade
!pip install transformers
!pip install datasets
!pip install tokenizers

     |████████████████████████████████| 2.6 MB 11.0 MB/s 
     |████████████████████████████████| 3.3 MB 34.7 MB/s 
     |████████████████████████████████| 636 kB 40.5 MB/s 
     |████████████████████████████████| 895 kB 36.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 13.2 MB/s 
     |████████████████████████████████| 243 kB 46.2 MB/s 
     |████████████████████████████████| 118 kB 50.3 MB/s 


###### *Data handling examples*

from https://medium.com/@prajwal.prashanth22/google-colab-drive-as-persistent-storage-for-long-training-runs-cb82bc1d5b71

In [ ]:
!wget -c https://github.com/lucidrains/h-transformer-1d/raw/main/data/enwik8.gz -P ./data/

In [ ]:
!unzip -qq '/content/gdrive/My Drive/Sample/sample'

In [ ]:
from keras.callbacks import *
filepath = "/content/gdrive/My Drive/MyCNN/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.load_weights('/content/gdrive/My Drive/MyCNN/epochs:047-val_acc:0.905.hdf5')

---
##### **DATA**

In [2]:
!pip install datasets

In [3]:
from datasets import load_dataset
# dataset = load_dataset('oscar', 'unshuffled_deduplicated_no', split='train[:10%]')
dataset = load_dataset('oscar', 'unshuffled_deduplicated_nds')  # low german

Downloading:   0%|          | 0.00/5.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/359k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.27M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset oscar downloaded and prepared to /root/.cache/huggingface/datasets/oscar/unshuffled_deduplicated_nds/1.0.0/84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2. Subsequent calls will reuse this data.


In [4]:


from tqdm.auto import tqdm

text_data = []
file_count = 0
!mkdir data
for sample in tqdm(dataset['train']):
    sample = sample['text'].replace('\n', '')
    text_data.append(sample)
    if len(text_data) == 10_000:
        # once we git the 10K mark, save to file
        with open(f'./data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 10K chunks, we will have ~2082 leftover samples, we save those now too
with open(f'./data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/8714 [00:00<?, ?it/s]

###### *explore dataset:*


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 8714
    })
})

In [ ]:
dataset['train']

Dataset({
    features: ['id', 'text'],
    num_rows: 8714
})

In [ ]:
dataset['train'].features

{'id': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None)}

In [ ]:
dataset['train'][0]

{'id': 0,
 'text': 'Dor kann sik vun nu af an de hele plattdüütsche Welt – vun Niebüll bit New York, vun Helgoland bit Honolulu – drapen. Allens, wat de NDR op Platt paraat hett, is hier to lesen, to hören un antokieken: Vun den Radioklassiker „Hör mal´n beten to“ bit to de Sennungen vun de Fernsehlüüd „Op platt“ (wat fröher maal Talk op platt weer) un „Nües ut Büttenwarder“ mit Jan Fedder und Klaus-Peter Brix is allens op enen Blick dorbi, wat Rang un Namen hett. Wat sünnerlich intressant is: Liekerveel of Hamborg or Namibia - Plattsnacker öwerall üm unsen Globus köönt sik rigistreern laten ünner http://www.ndr.de/platt/, wat öwer sik sülben vertellen or een Foto vun jüm in´t Nett setten. Se köönt denn op de nüe Platt-Weltkaart inteknen, woneem se tohuus sünd un sodennig anner Plattsnackers as Frünnen finnen un sik mit jüm feine „Mehls“ schriewen.\nFör de „Jusers“ gifft dat opto noch ´n Barg anner Hülp un Anbott. Se köönt geern för dat plattdüütsch Spraakbook ´n nü Woort tostüren, wat

---
##### **TOKENIZER**

In [ ]:
!pip install transformers
!pip install tokenizers

In [5]:
# TRAIN AND SAVE TOKENIZER

from pathlib import Path
paths = [str(x) for x in Path('./data').glob('**/*.txt')]


from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files=paths[:5], vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

!mkdir tokenizer
tokenizer.save_model('tokenizer')

# At this point the tokenizer is a bare tokenizer without any preprocessing or special tokens:
  # tokenizer: Tokenizer(vocabulary_size=30522, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)
# Only by loading it as a RobertaTokenizer, it remembers the saved special tokens.

['tokenizer/vocab.json', 'tokenizer/merges.txt']

In [6]:
# INITIALIZE TOKENIZER

from transformers import RobertaTokenizer

# initialize the tokenizer using the tokenizer we initialized and saved to file
tokenizer = RobertaTokenizer.from_pretrained('tokenizer', max_len=512)

# Now tokenizer is a:
  # PreTrainedTokenizer(name_or_path='tokenizer', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})

file tokenizer/config.json not found


###### *test tokenizer*

In [80]:
tokenizer

Tokenizer(vocabulary_size=30522, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)

In [ ]:
tokenizer("My name is Clemens.")

In [ ]:
tokenizer("My name is Clemens.").input_ids

In [67]:
tokenizer.encode("My name is Clemens.")

[0, 23281, 458, 2706, 326, 19446, 18, 2]

In [71]:
tokenizer.decode([0, 23281, 458, 2706, 326, 19446, 18, 2])

'<s>My name is Clemens.</s>'

---
##### **DATA PREP**

In [7]:
with open('./data/text_0.txt', 'r', encoding='utf-8') as fp:
    lines = fp.read().split('\n')

In [ ]:
batch = tokenizer(lines, max_length=512, padding='max_length', truncation=True)
# batch.keys(): dict_keys(['input_ids', 'attention_mask'])
len(batch['input_ids'])

In [20]:
# MLM PREP

import torch

labels = torch.tensor([x for x in batch['input_ids']])
mask = torch.tensor([x for x in batch['attention_mask']])

# make copy of labels tensor, this will be input_ids
input_ids = labels.detach().clone()
# create random array of floats with equal dims to input_ids
rand = torch.rand(input_ids.shape)
# mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)
# loop through each row in input_ids tensor (cannot do in parallel)
for i in range(input_ids.shape[0]):
    # get indices of mask positions from mask array
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    # mask input_ids
    input_ids[i, selection] = 3  # our custom [MASK] token == 3

# to explore the resulting tensors, see next subsection

In [21]:
# BUILDING THE DATALOADER

encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels}

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

dataset = Dataset(encodings)
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

###### *explore tensors*

In [ ]:
input_ids.shape

In [ ]:
input_ids[0][:200]

---
##### **MODEL**

In [ ]:
!pip install h-transformer-1d --upgrade

In [ ]:
from h_transformer_1d import HTransformer1D
from h_transformer_1d.autoregressive_wrapper import AutoregressiveWrapper

import random
import tqdm
import gzip
import numpy as np
import torch
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

# constants

NUM_BATCHES = int(1e5)
BATCH_SIZE = 4
GRADIENT_ACCUMULATE_EVERY = 4
LEARNING_RATE = 2e-4
VALIDATE_EVERY  = 100
GENERATE_EVERY  = 500
GENERATE_LENGTH = 512
SEQ_LEN = 4096

# helpers

def cycle(loader):
    while True:
        for data in loader:
            yield data

def decode_token(token):
    return str(chr(max(32, token)))

def decode_tokens(tokens):
    return ''.join(list(map(decode_token, tokens)))

# instantiate GPT-like decoder model

model = HTransformer1D(
    num_tokens = 256,
    dim = 512,
    max_seq_len = SEQ_LEN,
    depth = 8,
    heads = 8,
    causal = True,
    reversible = True
)

model = AutoregressiveWrapper(model)
model.cuda()

# prepare enwik8 data

with gzip.open('./data/enwik8.gz') as file:
    X = np.fromstring(file.read(int(95e6)), dtype=np.uint8)
    trX, vaX = np.split(X, [int(90e6)])
    data_train, data_val = torch.from_numpy(trX), torch.from_numpy(vaX)

class TextSamplerDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = seq_len

    def __getitem__(self, index):
        rand_start = torch.randint(0, self.data.size(0) - self.seq_len, (1,))
        full_seq = self.data[rand_start: rand_start + self.seq_len + 1].long()
        return full_seq.cuda()

    def __len__(self):
        return self.data.size(0) // self.seq_len

train_dataset = TextSamplerDataset(data_train, SEQ_LEN)
val_dataset   = TextSamplerDataset(data_val, SEQ_LEN)
train_loader  = cycle(DataLoader(train_dataset, batch_size = BATCH_SIZE))
val_loader    = cycle(DataLoader(val_dataset, batch_size = BATCH_SIZE))

# optimizer

optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# training

for i in tqdm.tqdm(range(NUM_BATCHES), mininterval=10., desc='training'):
    model.train()

    for __ in range(GRADIENT_ACCUMULATE_EVERY):
        loss = model(next(train_loader))
        loss.backward()

    print("training loss: {:.4f}".format(loss.item()))
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
    optim.step()
    optim.zero_grad()

    if i % VALIDATE_EVERY == 0:
        model.eval()
        with torch.no_grad():
            loss = model(next(val_loader))
            print(f'validation loss: {loss.item()}')

    if i % GENERATE_EVERY == 0:
        model.eval()
        inp = random.choice(val_dataset)[:-1]
        prime = decode_tokens(inp)
        print('{} \n {} \n'.format(prime, '*' * 100))

        sample = model.generate(inp, GENERATE_LENGTH)
        output_str = decode_tokens(sample)
        print(output_str)


---
##### **SOURCES**

1.   DATA
*   https://towardsdatascience.com/how-to-train-a-bert-model-from-scratch-72cfce554fc6
---
2.   TOKENIZER
* https://gist.github.com/aditya-malte/2d4f896f471be9c38eb4d723a710768b#file-smallberta_pretraining-ipynb
* [Data:1]
* https://huggingface.co/blog/how-to-train
---
3.   MODEL
* https://github.com/lucidrains/h-transformer-1d/blob/main/train.py